In [1]:
# Import necessary libraries
import librosa
import numpy as np
import pandas as pd
import os
import re
from scipy.stats import pearsonr
from pyramidi import models

# Import the two folders
print("Please copy and past the destination of the first folder that you want to analyze:")
input_folder1 = input()
print("Please write whether these are 'stimuli' or 'full tracks':")
input_type1 = input()
print("Please copy and past the destination of the second folder that you want to analyze:")
input_folder2 = input()
print("Please write whether these are 'stimuli' or 'full tracks':")
input_type2 = input()
print("Please wait around 30 seconds...")

def detect_mode(chroma_profile):
    """
    Detect the musical mode (Major/Minor) based on the chroma profile using MIRmode.
    
    Args:
        chroma_profile (numpy.ndarray): The chroma profile of the audio.
    
    Returns:
        str: The detected mode ('Major' or 'Minor').
    """
    return models.mirmode(chroma_profile, weights="BellmanBudge")

def get_pitch_height_and_std(y, sr):
    """
    Calculate the average pitch height (in MIDI note numbers) and its standard deviation
    based on the audio's Constant-Q Transform (CQT).

    Args:
        y (numpy.ndarray): The audio time-series data.
        sr (int): The sampling rate of the audio.

    Returns:
        tuple: The weighted average pitch height and the standard deviation of pitch height.
    """
    # Compute the constant-Q transform (CQT) for pitch extraction
    cqt_pitches = np.abs(librosa.cqt(y, sr=sr, n_bins=88, bins_per_octave=12, fmin=librosa.note_to_hz('A0')))
    
    # Sum energy across time for each key (MIDI note)
    key_energy = np.sum(cqt_pitches, axis=1)
    # Get MIDI note numbers for 88 keys (A0 to C8)
    keyboard_notes = np.arange(21, 109)

    # Calculate the weighted average pitch height (in MIDI numbers)
    average_pitch = np.sum(key_energy * keyboard_notes) / np.sum(key_energy)
    
    return average_pitch

def remove_last_two_seconds(file_path, sampling_rate=44100):
    """
    Load an audio file using both Librosa and Essentia, and remove the last two seconds.

    Parameters:
    -----------
    file_path : str
        Path to the audio file to be loaded.
    sampling_rate : int, optional
        Sampling rate for loading the audio file. Default is 44100 Hz.

    Returns:
    --------
    librosa_audio : numpy.ndarray
        The audio signal without the last two seconds (Librosa).
    essentia_audio : numpy.ndarray
        The audio signal without the last two seconds (Essentia).
    librosa_duration : float
        The duration (in seconds) of the trimmed audio from Librosa.
    essentia_duration : float
        The duration (in seconds) of the trimmed audio from Essentia.
    """
    # Load the audio file with Librosa
    y, sr = librosa.load(file_path, sr=sampling_rate)
    
    # Calculate the number of samples corresponding to the last 2 seconds
    num_samples_to_remove = int(2 * sr)
    
    # Remove the last 2 seconds of the Librosa audio
    librosa_audio = y[:-num_samples_to_remove] if len(y) > num_samples_to_remove else []

    return librosa_audio, sr

def extract_audio_features(input_folder, type):
    """
    Extract various audio features from the audio files in a specified folder.
    
    Args:
        input_folder (str): Path to the folder containing audio files.
    
    Returns:
        tuple: Contains the pitch class distributions, tempos, attack rates,
               modes, average volumes, and average pitch heights.
    """
    # Dictionaries to store extracted features
    attack_rates = dict()
    modes = dict()
    average_pitch_heights = dict()

    # Iterate over files in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.wav', '.mp3', '.flac', '.aiff')):
            file_path = os.path.join(input_folder, filename)
            match = re.search(r'(\d{1,2})\s*[_]*\s*(Major|minor)', filename)
            
            # Extract piece number and tonality from filename
            if match:
                piece_number = match.group(1)
                tonality = match.group(2)
                piece_name = f"{piece_number} {tonality}"
            else:
                piece_name = 'Unknown'

            if type == "stimuli":
                # Load the audio using librosa
                y, sr = remove_last_two_seconds(file_path)
            else:
                y, sr = librosa.load(file_path, sr=44100)

            # Get the duration of the audio
            duration = librosa.get_duration(y=y, sr=sr)

            # Calculate the average pitch height of the audio
            pitch_height = get_pitch_height_and_std(y, sr)

            # Detect the musical mode based on the chroma profile
            # Extract chroma features and calculate the average chroma profile
            chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
            avg_chroma_profile = np.mean(chroma, axis=1)
            avg_pitch_class_distribution = list(avg_chroma_profile)
            
            mode = detect_mode(avg_pitch_class_distribution)

            # Calculate the number of onsets and compute the attack rate
            num_onsets = len(list(librosa.onset.onset_detect(y=y, sr=sr, units='time')))
            attack_rate = num_onsets / duration
            
            # Store the extracted features in the respective dictionaries
            attack_rates[piece_name] = attack_rate
            average_pitch_heights[piece_name] = pitch_height
            modes[piece_name] = mode

    return attack_rates, modes, average_pitch_heights

def compute_dict_correlation(dict1, dict2):
    """
    Compute the Pearson correlation between values of two dictionaries for their common keys.
    
    Args:
        dict1 (dict): The first dictionary.
        dict2 (dict): The second dictionary.
    
    Returns:
        tuple: Pearson correlation coefficient and p-value.
    """
    common_keys = set(dict1.keys()).intersection(dict2.keys())
    values1 = [dict1[key] for key in common_keys]
    values2 = [dict2[key] for key in common_keys]
    
    # Calculate and return the Pearson correlation
    return pearsonr(values1, values2)

# Extract features from both input folders
ar1, modes1, average_pitch_heights1 = extract_audio_features(input_folder1, input_type1)
ar2, modes2, average_pitch_heights2 = extract_audio_features(input_folder2, input_type2)

# Calculate Pearson correlations for other features
attack_rate_correlation, _ = compute_dict_correlation(ar1, ar2)
ph_correlation, _ = compute_dict_correlation(average_pitch_heights1, average_pitch_heights2)
mode_correlation, _ = compute_dict_correlation(modes1, modes2)

# Print correlation results for various features
print("Correlation of Attack Rates:", round(attack_rate_correlation, 3))
print("Correlation of Pitch Heights:", round(ph_correlation, 3))
print("Correlation of Modes:", round(mode_correlation, 3))

Please copy and past the destination of the first folder that you want to analyze:
Please write whether these are 'stimuli' or 'full tracks':
Please copy and past the destination of the second folder that you want to analyze:
Please write whether these are 'stimuli' or 'full tracks':
Please wait around 30 seconds...
Correlation of Attack Rates: 0.903
Correlation of Pitch Heights: 0.68
Correlation of Modes: 0.874


Scriabin

Stimuli:
'/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Scriabin/scriabinLevy2015 - Stimuli'

Full Tracks:
'/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Scriabin/scriabinLevy2015 - Full Tracks'

Shostakovich

Stimuli:
'/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Shostakovich/shostakovichBoyadjieva2009 - Stimuli'

Full Tracks:
'/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Shostakovich/shostakovichBoyadjieva2009 - Full Tracks'

Kabalevsky

Stimuli: '/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Kabalevksy/kabalevskyDossin2007 - Stimuli'

Full Tracks: '/Users/Ben/Downloads/MAPLE Lab/Thesis_Project/Audio Files/Kabalevksy/kabalevskyDossin2007 - Full Tracks'



In [31]:
import csv
ar1 = dict(sorted(ar1.items()))
ar2 = dict(sorted(ar2.items()))

# Combine dictionaries by keys
combined_data = [["PieceID", "Stimuli", "Full Tracks", "Mode"]]  # Add a header row
for key in ar1.keys():
    value1 = ar1.get(key, "")  # Get value from ar1 or default to empty string
    value2 = ar2.get(key, "")  # Get value from ar2 or default to empty string
    
    # Determine the mode (Major or minor)
    if "M" in key:
        mode = "Major"
    else:
        mode = "minor"

    combined_data.append([key, value1, value2, mode])

# Write to CSV
with open('ar_st_ka.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(combined_data)

print("CSV file has been created.")


CSV file has been created.


In [32]:
import csv
average_pitch_heights1 = dict(sorted(average_pitch_heights1.items()))
average_pitch_heights2 = dict(sorted(average_pitch_heights2.items()))

# Combine dictionaries by keys
combined_data = [["PieceID", "Stimuli", "Full Tracks", "Mode"]]  # Add a header row
for key in average_pitch_heights1.keys():
    value1 = average_pitch_heights1.get(key, "")  # Get value from tempos1 or default to empty string
    value2 = average_pitch_heights2.get(key, "")  # Get value from tempos2 or default to empty string
    if "M" in key:
        mode = "Major"
    else:
        mode = "minor"

    combined_data.append([key, value1, value2, mode])

# Write to CSV
with open('ph_st_ka.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(combined_data)

print("CSV file has been created.")

CSV file has been created.


In [33]:
import csv
modes1 = dict(sorted(modes1.items()))
modes2 = dict(sorted(modes2.items()))

# Combine dictionaries by keys
combined_data = [["PieceID", "Stimuli", "Full Tracks", "Mode"]]  # Add a header row
for key in modes1.keys():
    value1 = modes1.get(key, "")  # Get value from tempos1 or default to empty string
    value2 = modes2.get(key, "")  # Get value from tempos2 or default to empty string
    if "M" in key:
        mode = "Major"
    else:
        mode = "minor"

    combined_data.append([key, value1, value2, mode])

# Write to CSV
with open('mode_st_ka.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(combined_data)

print("CSV file has been created.")

CSV file has been created.
